In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
import pandas as pd, numpy as np

In [16]:
username='donttellthewarden'

browser  = webdriver.Firefox()
browser.get('https://www.instagram.com/'+username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [33]:
links=[]
source = browser.page_source
data=bs(source, 'html.parser')
posts = data.find_all('a', class_='_a6hd')

In [34]:
print(posts)

[<a aria-selected="true" class="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _aa-z _ap3g _aa-- _ac_u _a6hd" href="/donttellthewarden/?hl=en" role="tab" tabindex="0"><div class="x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1n2onr6 x1plvlek xryxfnj x1c4vz4f x2lah0s x1q0g3np xqjyukv x6s0dn4 x1oa3qoh x1nhvcw1"><svg aria-label="" class="x1lliihq x1n2onr6 x5n08af" fill="currentColor" height="12" role="img" viewbox="0 0 24 24" width="12"><title></title><rect fill="none" height="18" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" width="18" x="3" y="3"></rect><line fill="none" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" x1="9.015" x2="9.015" y1="3" y2="21"></line><line fill="none" stroke="currentColor" stroke-linecap="round" stroke-

In [63]:
links = []

In [64]:
for post in posts:
    link = "https://instagram.com" + post.get('href')
    links.append(link)

In [65]:
print("Links:", links)

Links: ['https://instagram.com/donttellthewarden/?hl=en', 'https://instagram.com/donttellthewarden/reels/?hl=en', 'https://instagram.com/accounts/login/?next=%2Fdonttellthewarden%2Ftagged%2F&source=profile_tagged_tab&hl=en', 'https://instagram.com/donttellthewarden/p/C8FGjTbvd2Q/?hl=en', 'https://instagram.com/donttellthewarden/p/C7rai1IiVI4/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ozo7Ev81v/?hl=en', 'https://instagram.com/donttellthewarden/reel/C7oQIk7CCVJ/?hl=en', 'https://instagram.com/donttellthewarden/p/C7mE2Yxv5JS/?hl=en', 'https://instagram.com/donttellthewarden/reel/C7jW0T7v1UR/?hl=en', 'https://instagram.com/donttellthewarden/p/C7g_7liPRJq/?hl=en', 'https://instagram.com/donttellthewarden/p/C7f7dVmvtHI/?hl=en', 'https://instagram.com/donttellthewarden/p/C7bvl3uigeT/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ZW3TACeMS/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ToovhvLDM/?hl=en', 'https://instagram.com/donttellthewarden/p/C7HcEfVPwA4/?hl=en', '

In [66]:
links = links[3:-7]

In [67]:
print("Links:", links)

Links: ['https://instagram.com/donttellthewarden/p/C8FGjTbvd2Q/?hl=en', 'https://instagram.com/donttellthewarden/p/C7rai1IiVI4/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ozo7Ev81v/?hl=en', 'https://instagram.com/donttellthewarden/reel/C7oQIk7CCVJ/?hl=en', 'https://instagram.com/donttellthewarden/p/C7mE2Yxv5JS/?hl=en', 'https://instagram.com/donttellthewarden/reel/C7jW0T7v1UR/?hl=en', 'https://instagram.com/donttellthewarden/p/C7g_7liPRJq/?hl=en', 'https://instagram.com/donttellthewarden/p/C7f7dVmvtHI/?hl=en', 'https://instagram.com/donttellthewarden/p/C7bvl3uigeT/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ZW3TACeMS/?hl=en', 'https://instagram.com/donttellthewarden/p/C7ToovhvLDM/?hl=en', 'https://instagram.com/donttellthewarden/p/C7HcEfVPwA4/?hl=en']


# Login

In [108]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [109]:
browser  = webdriver.Firefox()
browser.get('https://www.instagram.com/')

In [110]:
WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']")))
username = browser.find_element(By.CSS_SELECTOR, "input[name='username']")
password = browser.find_element(By.CSS_SELECTOR, "input[name='password']")
username.clear()
password.clear()
username.send_keys("ithinktheytoldthewarden")
password.send_keys("lndlndlnd")

In [111]:
login_button = browser.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_button.click()

In [ ]:
alert = WebDriverWait(browser, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

In [112]:
username='donttellthewarden'
browser.get('https://www.instagram.com/'+username+'/?hl=en')
links = set()

In [113]:
n_scrolls = 15
for _ in range(n_scrolls):
    source = browser.page_source
    data=bs(source, 'html.parser')
    posts = data.find_all('a', class_='_a6hd')
    
    for post in posts:
        href = post.get('href')
        if href.startswith('/p/') or href.startswith('/reels/'):
            link = "https://instagram.com" + href
            links.add(link)
            
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(2)

In [116]:
print(links)

{'https://instagram.com/p/CVxmEKAPLKo/?hl=en', 'https://instagram.com/p/C7mE2Yxv5JS/?hl=en', 'https://instagram.com/p/CQqvOgCMUye/?hl=en', 'https://instagram.com/p/CVsP373vkMm/?hl=en', 'https://instagram.com/p/CByGNmpjFSY/?hl=en', 'https://instagram.com/p/Ct9Zmodsk-1/?hl=en', 'https://instagram.com/p/C2KM8Vov7-N/?hl=en', 'https://instagram.com/p/CJEBX9PDnZT/?hl=en', 'https://instagram.com/p/CO7gMuAjAPz/?hl=en', 'https://instagram.com/p/CJWGONaD8St/?hl=en', 'https://instagram.com/p/CB12LZBDU-r/?hl=en', 'https://instagram.com/p/C4aqJIdvE1N/?hl=en', 'https://instagram.com/p/CsqtpaWvBGF/?hl=en', 'https://instagram.com/p/CbkHMknPQO6/?hl=en', 'https://instagram.com/p/CU5JflxPqxO/?hl=en', 'https://instagram.com/p/CGzxQi6D8-X/?hl=en', 'https://instagram.com/p/Cjx4RuLPdkQ/?hl=en', 'https://instagram.com/p/CXtcYmrvGse/?hl=en', 'https://instagram.com/p/CAdEznuDaEz/?hl=en', 'https://instagram.com/p/CPx8GCsDTJy/?hl=en', 'https://instagram.com/p/Ct__UPEPFEs/?hl=en', 'https://instagram.com/p/CfjLiNmB

In [117]:
print(len(links))

174
